In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.dates as mdates
from typing import List, Dict, Optional

# 全局配置
pd.set_option('display.max_columns', None)
plt.rcParams["font.family"] = ["SimHei", "WenQuanYi Micro Hei", "Heiti TC"]
plt.rcParams['axes.unicode_minus'] = False

# 基础必需字段（基于sample.csv和字段文档，覆盖6大核心维度）
BASE_REQUIRED_COLS = [
    # 1. 基础行情
    "date", "stock_code", "stock_name", "open", "close", "low", "high",
    "volume", "money", "pre_close", "high_limit", "low_limit", "paused",
    # 2. 竞价信息
    "auc_volume", "auc_money", "auc_volume_ratio_vs_5d_avg",
    # 3. 盘口数据
    "buy_total", "sell_total", "order_book_volume_ratio",
    # 4. 行业分类
    "sw_l1_industry_name", "sw_l2_industry_name",
    # 5. 估值与市值
    "turnover_ratio", "pe_ratio_lyr", "pb_ratio", "ps_ratio",
    # 6. 技术指标（含量能深度）
    "ma5", "ma10", "ma20", "rsi14", "macd_line", "signal_line", "macd_hist",
    "volatility", "amplitude", "momentum14", "volume_ratio_vs_5d_avg",
    "money_ratio_vs_5d_avg", "turnover_ratio_vs_5d_avg", "amplitude_vs_5d_avg"
]

class FullFieldStockSelector:
    def __init__(self, data_path: str, 
                 output_dir: str = "selected_stocks_full_field",
                 result_dir: str = "selection_results_full",
                 performance_dir: str = "strategy_performance_full"):
        """初始化全字段应用选股器"""
        self.data_path = data_path
        self.output_dir = output_dir
        self.result_dir = result_dir
        self.performance_dir = performance_dir
        
        # 创建输出目录
        os.makedirs(self.output_dir, exist_ok=True)
        os.makedirs(self.result_dir, exist_ok=True)
        os.makedirs(self.performance_dir, exist_ok=True)
        
        # 数据存储
        self.raw_data = None  # 原始完整数据
        self.processed_data = None  # 预处理后数据
        self.trading_dates = []  # 有效交易日列表
        
        # 结果存储
        self.all_selected = None  # 所有选股结果
        self.trade_records = None  # 交易记录
        self.overall_stats = None  # 策略整体统计

    def load_and_preprocess_data(self) -> bool:
        """加载并预处理全字段数据（无未来函数）"""
        try:
            # 1. 加载原始数据（支持CSV/Parquet，根据实际格式选择）
            print("加载全字段原始数据...")
            if self.data_path.endswith(".csv"):
                self.raw_data = pd.read_csv(self.data_path)
            elif self.data_path.endswith(".parquet"):
                self.raw_data = pd.read_parquet(self.data_path)
            else:
                raise ValueError("仅支持CSV或Parquet格式数据")
            
            print(f"原始数据规模：{len(self.raw_data)} 行 × {len(self.raw_data.columns)} 列")
            
            # 2. 检查必需字段
            missing_cols = [col for col in BASE_REQUIRED_COLS if col not in self.raw_data.columns]
            if missing_cols:
                raise ValueError(f"❌ 缺失必需字段：{missing_cols}")
            
            # 3. 数据格式处理
            df = self.raw_data[BASE_REQUIRED_COLS].copy()
            # 日期格式统一
            df["date"] = pd.to_datetime(df["date"])
            # 数值字段转换（避免字符串格式）
            numeric_cols = [col for col in BASE_REQUIRED_COLS if col not in ["date", "stock_code", "stock_name", "sw_l1_industry_name", "sw_l2_industry_name"]]
            for col in numeric_cols:
                df[col] = pd.to_numeric(df[col], errors="coerce")
            
            # 4. 缺失值处理（仅用历史数据填充，无未来函数）
            df = df.sort_values(["stock_code", "date"])
            # 数值字段按股票分组前向填充
            for col in numeric_cols:
                df[col] = df.groupby("stock_code")[col].ffill()
            # 行业字段填充
            df["sw_l1_industry_name"] = df.groupby("stock_code")["sw_l1_industry_name"].ffill()
            df["sw_l2_industry_name"] = df.groupby("stock_code")["sw_l2_industry_name"].ffill()
            
            # 5. 过滤无效数据（停牌、关键字段缺失、极低流动性）
            initial_count = len(df)
            df = df[
                (df["paused"] == 0) &  # 排除停牌股
                (df["auc_volume"] > 0) &  # 排除无竞价数据股
                (df["turnover_ratio"] > 0.1) &  # 排除极低流动性股（换手率<0.1%）
                (df["close"] > 0)  # 排除无效价格股
            ].dropna(subset=numeric_cols)
            
            self.processed_data = df
            print(f"数据预处理完成：保留 {len(self.processed_data)} 行（删除 {initial_count - len(self.processed_data)} 行无效数据）")
            
            # 6. 获取有效交易日
            self.trading_dates = sorted(self.processed_data["date"].unique())
            print(f"有效交易日范围：{self.trading_dates[0].strftime('%Y-%m-%d')} 至 {self.trading_dates[-1].strftime('%Y-%m-%d')}，共 {len(self.trading_dates)} 天")
            
            return True
        except Exception as e:
            print(f"数据处理失败：{str(e)}")
            return False

    def calculate_t_day_core_indicators(self, date_data: pd.DataFrame) -> pd.DataFrame:
        """计算T日核心衍生指标（无未来函数）"""
        df = date_data.copy()
        
        # 1. 当日收益率（开盘→收盘）
        df["t_day_return"] = (df["close"] - df["open"]) / df["open"]
        
        # 2. 价格趋势状态
        df["above_ma5"] = df["close"] > df["ma5"]  # 收盘价在5日均线上方
        df["ma5_up"] = df.groupby("stock_code")["ma5"].transform(lambda x: x > x.shift(1))  # 5日线向上（对比T-1）
        
        # 3. 量能匹配度（成交量与成交额同步性）
        df["volume_money_match"] = (df["volume_ratio_vs_5d_avg"] >= 1.1) & (df["money_ratio_vs_5d_avg"] >= 1.1)
        
        # 4. 盘口强弱（买盘-卖盘差比例）
        df["buy_sell_diff_ratio"] = (df["buy_total"] - df["sell_total"]) / df["sell_total"].clip(lower=1)  # 避免除零
        
        # 5. 估值安全度（静态PE>0且<50，PB<5）
        df["valuation_safe"] = (df["pe_ratio_lyr"] > 0) & (df["pe_ratio_lyr"] < 50) & (df["pb_ratio"] < 5)
        
        # 6. 波动控制（当日振幅<7%）
        df["low_amplitude"] = df["amplitude"] < 0.07
        
        return df

    def judge_market_sentiment(self, date: datetime) -> str:
        """基于全字段判断T日市场情绪（强/中/弱）"""
        date_data = self.processed_data[self.processed_data["date"] == date].copy()
        if date_data.empty:
            return "weak"  # 无数据视为弱情绪
        
        date_data = self.calculate_t_day_core_indicators(date_data)
        
        # 1. 市场广度（30分）：上涨占比+涨停数
        up_ratio = (date_data["t_day_return"] > 0).mean()
        limit_up_count = (date_data["close"] >= date_data["high_limit"] * 0.995).sum()
        breadth_score = 0
        breadth_score += 20 if up_ratio >= 0.6 else 10 if up_ratio >= 0.5 else 0
        breadth_score += 10 if limit_up_count >= 50 else 5 if limit_up_count >= 20 else 0
        
        # 2. 资金活跃度（30分）：平均换手率+竞价量比
        avg_turnover_ratio = date_data["turnover_ratio"].mean()
        avg_auc_volume_ratio = date_data["auc_volume_ratio_vs_5d_avg"].mean()
        activity_score = 0
        activity_score += 15 if avg_turnover_ratio >= 3.0 else 5 if avg_turnover_ratio >= 2.0 else 0
        activity_score += 15 if avg_auc_volume_ratio >= 1.2 else 5 if avg_auc_volume_ratio >= 1.0 else 0
        
        # 3. 趋势强度（40分）：均线以上比例+低波动比例
        above_ma5_ratio = date_data["above_ma5"].mean()
        low_volatility_ratio = (date_data["volatility"] < 0.05).mean()
        trend_score = 0
        trend_score += 20 if above_ma5_ratio >= 0.6 else 10 if above_ma5_ratio >= 0.5 else 0
        trend_score += 20 if low_volatility_ratio >= 0.7 else 10 if low_volatility_ratio >= 0.5 else 0
        
        # 总得分与情绪判定
        total_score = breadth_score + activity_score + trend_score
        if total_score >= 70:
            return "strong"
        elif total_score >= 40:
            return "neutral"
        else:
            return "weak"

    def select_strong_industries(self, date: datetime, top_n: int = 3) -> List[str]:
        """基于量能+竞价+估值筛选T日强行业"""
        date_data = self.processed_data[self.processed_data["date"] == date].copy()
        date_data = self.calculate_t_day_core_indicators(date_data)
        
        # 按申万一级行业分组计算核心指标
        industry_metrics = date_data.groupby("sw_l1_industry_name").agg({
            "t_day_return": "mean",  # 行业平均收益率
            "auc_volume_ratio_vs_5d_avg": "mean",  # 行业平均竞价量比
            "volume_money_match": "mean",  # 行业量能匹配度（0-1）
            "valuation_safe": "mean",  # 行业估值安全度（0-1）
            "low_amplitude": "mean"  # 行业低波动比例（0-1）
        }).reset_index()
        
        # 行业强度得分（标准化到0-100分，权重基于实盘有效性）
        # 收益率标准化（避免不同日期绝对值差异）
        max_return = industry_metrics["t_day_return"].max()
        industry_metrics["norm_return"] = industry_metrics["t_day_return"] / (max_return + 1e-8)  # 防除零
        
        industry_metrics["strength_score"] = (
            industry_metrics["norm_return"] * 40 +  # 收益能力（40%）
            industry_metrics["auc_volume_ratio_vs_5d_avg"].clip(0, 2) / 2 * 20 +  # 竞价热度（20%）
            industry_metrics["volume_money_match"] * 15 +  # 量能匹配（15%）
            industry_metrics["valuation_safe"] * 15 +  # 估值安全（15%）
            industry_metrics["low_amplitude"] * 10  # 波动控制（10%）
        )
        
        # 筛选有效行业（至少5只个股，避免小行业偶然性）
        industry_size = date_data.groupby("sw_l1_industry_name").size().reset_index(name="count")
        valid_industries = industry_size[industry_size["count"] >= 5]["sw_l1_industry_name"].tolist()
        filtered_industries = industry_metrics[industry_metrics["sw_l1_industry_name"].isin(valid_industries)]
        
        # 取TOP N强行业
        top_industries = filtered_industries.nlargest(top_n, "strength_score")["sw_l1_industry_name"].tolist()
        
        # 打印强行业结果
        print(f"\nT日强行业TOP{len(top_industries)}：")
        for idx, ind in enumerate(top_industries, 1):
            ind_data = filtered_industries[filtered_industries["sw_l1_industry_name"] == ind].iloc[0]
            print(f"  {idx}. {ind}：强度{ind_data['strength_score']:.1f}分 | 平均收益{ind_data['t_day_return']*100:.2f}% | 竞价量比{ind_data['auc_volume_ratio_vs_5d_avg']:.2f}")
        
        return top_industries

    def select_stocks_for_date(self, date: datetime) -> Optional[pd.DataFrame]:
        """T日选股（全字段多维度筛选+评分）"""
        # 1. 先做情绪过滤（弱情绪直接不选股）
        sentiment = self.judge_market_sentiment(date)
        if sentiment == "weak":
            print(f"T日({date.strftime('%Y-%m-%d')}) 情绪：弱（市场环境差），不选股")
            return None
        print(f"\nT日({date.strftime('%Y-%m-%d')}) 情绪：{sentiment.upper()}（适合选股）")
        
        # 2. 筛选T日强行业（仅在强行业内选股）
        strong_industries = self.select_strong_industries(date)
        if not strong_industries:
            print("无符合条件的强行业（行业个股数不足或强度不达标），不选股")
            return None
        
        # 3. 获取T日强行业个股数据并计算核心指标
        date_data = self.processed_data[
            (self.processed_data["date"] == date) &
            (self.processed_data["sw_l1_industry_name"].isin(strong_industries))
        ].copy()
        date_data = self.calculate_t_day_core_indicators(date_data)
        
        # 4. 个股多维度硬性筛选（排除明显风险股）
        filtered_stocks = date_data[
            # 估值安全（避免高估值回调）
            (date_data["valuation_safe"]) &
            # 竞价承接（早盘资金关注）
            (date_data["auc_volume_ratio_vs_5d_avg"] >= 1.2) &
            # 量能充足（日内资金持续）
            (date_data["volume_ratio_vs_5d_avg"] >= 1.1) &
            # 盘口强势（实时多空平衡）
            (date_data["order_book_volume_ratio"] >= 1.2) &  # 买盘/卖盘≥1.2
            (date_data["buy_sell_diff_ratio"] > 0) &  # 买盘>卖盘
            # 趋势稳健（避免短期反转）
            (date_data["above_ma5"]) &  # 收盘价在5日均线上
            (date_data["volatility"] < 0.05) &  # 20日波动率<5%（低波动）
            (date_data["momentum14"] > 0) &  # 14日动量>0（趋势向上）
            (date_data["low_amplitude"])  # 当日振幅<7%（日内稳定）
        ].copy()
        
        if len(filtered_stocks) == 0:
            print("强行业内无符合所有筛选条件的个股，不选股")
            return None
        
        # 5. 个股综合评分（100分制，量化选股优先级）
        # 5.1 趋势得分（20分）：均线+动量
        filtered_stocks["trend_score"] = 0
        filtered_stocks.loc[filtered_stocks["ma5_up"], "trend_score"] += 10  # 5日线向上
        filtered_stocks.loc[filtered_stocks["momentum14"] > 0.03, "trend_score"] += 10  # 14日动量>3%
        
        # 5.2 资金得分（30分）：竞价+成交量+成交额
        filtered_stocks["capital_score"] = 0
        filtered_stocks.loc[filtered_stocks["auc_volume_ratio_vs_5d_avg"] >= 1.3, "capital_score"] += 10  # 竞价量比高
        filtered_stocks.loc[filtered_stocks["volume_ratio_vs_5d_avg"] >= 1.3, "capital_score"] += 10  # 成交量比高
        filtered_stocks.loc[filtered_stocks["money_ratio_vs_5d_avg"] >= 1.2, "capital_score"] += 10  # 成交额比高
        
        # 5.3 估值得分（20分）：PE+PB
        filtered_stocks["valuation_score"] = 0
        filtered_stocks.loc[filtered_stocks["pe_ratio_lyr"] < 30, "valuation_score"] += 10  # 静态PE<30
        filtered_stocks.loc[filtered_stocks["pb_ratio"] < 3, "valuation_score"] += 10  # PB<3
        
        # 5.4 盘口得分（10分）：量比+买卖差
        filtered_stocks["market_score"] = 0
        filtered_stocks.loc[filtered_stocks["order_book_volume_ratio"] >= 1.5, "market_score"] += 5  # 盘口量比高
        filtered_stocks.loc[filtered_stocks["buy_sell_diff_ratio"] > 0.2, "market_score"] += 5  # 买卖差大
        
        # 5.5 收益得分（20分）：当日收益率标准化
        max_daily_return = filtered_stocks["t_day_return"].max()
        filtered_stocks["return_score"] = (filtered_stocks["t_day_return"] / (max_daily_return + 1e-8)) * 20
        
        # 5.6 总得分
        filtered_stocks["total_score"] = (
            filtered_stocks["trend_score"] +
            filtered_stocks["capital_score"] +
            filtered_stocks["valuation_score"] +
            filtered_stocks["market_score"] +
            filtered_stocks["return_score"]
        )
        
        # 6. 按行业取前2只（控制集中度，避免单行业风险）
        selected_stocks = []
        for industry in strong_industries:
            industry_stocks = filtered_stocks[filtered_stocks["sw_l1_industry_name"] == industry]
            if len(industry_stocks) == 0:
                continue
            # 按总得分排序，取前2只
            top2_stocks = industry_stocks.nlargest(2, "total_score")
            selected_stocks.append(top2_stocks)
        
        # 合并最终结果
        final_selected = pd.concat(selected_stocks, ignore_index=True)
        # 添加选股元信息
        final_selected["selection_date"] = date
        final_selected["market_sentiment"] = sentiment
        
        # 7. 格式化输出字段（便于查看）
        output_cols = [
            "selection_date", "stock_code", "stock_name", "sw_l1_industry_name",
            "t_day_return", "turnover_ratio", "auc_volume_ratio_vs_5d_avg",
            "order_book_volume_ratio", "pe_ratio_lyr", "pb_ratio",
            "trend_score", "capital_score", "valuation_score", "market_score", "return_score", "total_score"
        ]
        result_df = final_selected[output_cols].copy()
        
        # 字段格式化（百分比/小数保留）
        result_df["t_day_return"] = (result_df["t_day_return"] * 100).round(2)  # 转为百分比
        result_df["auc_volume_ratio_vs_5d_avg"] = result_df["auc_volume_ratio_vs_5d_avg"].round(2)
        result_df["order_book_volume_ratio"] = result_df["order_book_volume_ratio"].round(2)
        result_df["pe_ratio_lyr"] = result_df["pe_ratio_lyr"].round(2)
        result_df["pb_ratio"] = result_df["pb_ratio"].round(2)
        result_df["total_score"] = result_df["total_score"].round(1)
        
        # 重命名列名（更直观）
        result_df.columns = [
            "选股日期", "股票代码", "股票名称", "所属行业",
            "T日收益率(%)", "换手率(%)", "竞价量5日比",
            "盘口量比", "静态PE", "市净率PB",
            "趋势得分", "资金得分", "估值得分", "盘口得分", "收益得分", "综合评分"
        ]
        
        return result_df

    def run_daily_selection(self, start_date: Optional[str] = None, end_date: Optional[str] = None) -> bool:
        """按日期范围执行每日选股（批量处理）"""
        # 确定目标日期范围
        if start_date:
            start_date = pd.to_datetime(start_date)
            target_dates = [d for d in self.trading_dates if d >= start_date]
        else:
            target_dates = self.trading_dates
        
        if end_date:
            end_date = pd.to_datetime(end_date)
            target_dates = [d for d in target_dates if d <= end_date]
        
        if not target_dates:
            print("无符合条件的交易日（日期范围超出数据覆盖）")
            return False
        
        print(f"\n=== 开始按日选股 ===")
        print(f"日期范围：{target_dates[0].strftime('%Y-%m-%d')} 至 {target_dates[-1].strftime('%Y-%m-%d')}")
        print(f"总交易日数：{len(target_dates)} 天")
        
        # 存储所有选股结果
        all_results = []
        for date in tqdm(target_dates, desc="选股进度"):
            try:
                daily_result = self.select_stocks_for_date(date)
                if daily_result is not None and not daily_result.empty:
                    # 保存每日选股结果（按日期命名）
                    date_str = date.strftime("%Y%m%d")
                    daily_save_path = os.path.join(self.output_dir, f"selected_stocks_{date_str}.csv")
                    daily_result.to_csv(daily_save_path, index=False, encoding="utf-8-sig")
                    all_results.append(daily_result)
            except Exception as e:
                print(f"⚠️ {date.strftime('%Y-%m-%d')} 选股出错：{str(e)}（跳过该日）")
                continue
        
        # 合并所有结果并保存
        if all_results:
            self.all_selected = pd.concat(all_results, ignore_index=True)
            all_save_path = os.path.join(self.output_dir, "all_selected_stocks.csv")
            self.all_selected.to_csv(all_save_path, index=False, encoding="utf-8-sig")
            
            print(f"\n=== 选股完成 ===")
            print(f"总推荐个股数：{len(self.all_selected)} 只")
            print(f"平均每日推荐：{len(self.all_selected)/len(target_dates):.1f} 只")
            print(f"选股结果保存目录：{self.output_dir}")
            return True
        else:
            print(f"\n=== 选股完成 ===")
            print(f"未生成任何选股结果（可能所有日期均无符合条件个股）")
            return False

    def get_future_price(self, stock_code: str, base_date: datetime, days: int, price_type: str = "open") -> Optional[float]:
        """获取T+days日的价格（用于回测，严格无未来函数）"""
        try:
            # 1. 获取该股票的所有有效交易日（按时间排序）
            stock_data = self.processed_data[self.processed_data["stock_code"] == stock_code].sort_values("date")
            stock_dates = stock_data["date"].tolist()
            
            # 2. 找到基准日期（T日）在股票日期列表中的索引
            if base_date not in stock_dates:
                return None  # T日该股票无数据
            base_idx = stock_dates.index(base_date)
            
            # 3. 计算目标日期索引（T+days），确保不超出范围
            target_idx = base_idx + days
            if target_idx >= len(stock_dates):
                return None  # 无T+days日数据
            
            # 4. 获取目标日期的指定价格
            target_date = stock_dates[target_idx]
            target_price = stock_data[stock_data["date"] == target_date][price_type].values[0]
            
            return target_price
        except (ValueError, IndexError, KeyError):
            # 捕获索引错误、值错误（日期不在列表）、键错误（价格字段不存在）
            return None

    def calculate_strategy_returns(self) -> bool:
        """计算策略收益：T日选股→T+1竞价筛选→T+1/T+2开盘买入→T+3收盘卖出"""
        if self.all_selected is None or self.all_selected.empty:
            print("无选股结果，无法计算策略收益")
            return False
        
        # 确保选股日期格式正确
        self.all_selected["选股日期"] = pd.to_datetime(self.all_selected["选股日期"])
        trade_records = []
        
        print(f"\n=== 开始计算策略收益 ===")
        print(f"回测规则：T日选股 → T+1竞价筛选 → T+1/T+2开盘各买50% → T+3收盘卖出")
        
        # 遍历所有推荐个股，模拟交易
        for _, stock in tqdm(self.all_selected.iterrows(), desc="计算交易收益", total=len(self.all_selected)):
            stock_code = stock["股票代码"]
            stock_name = stock["股票名称"]
            select_date = stock["选股日期"]
            industry = stock["所属行业"]
            
            # 1. 获取关键价格数据（T+1/T+2开盘价，T+3收盘价）
            t1_open = self.get_future_price(stock_code, select_date, 1, "open")  # T+1开盘价（第一次买入）
            t2_open = self.get_future_price(stock_code, select_date, 2, "open")  # T+2开盘价（第二次买入）
            t3_close = self.get_future_price(stock_code, select_date, 3, "close")  # T+3收盘价（卖出）
            
            # 2. 获取T+1日竞价数据（用于买入前二次筛选）
            t1_date = select_date + timedelta(days=1)
            t1_auc_data = self.processed_data[
                (self.processed_data["stock_code"] == stock_code) &
                (self.processed_data["date"] == t1_date)
            ]
            if t1_auc_data.empty:
                continue  # T+1日无竞价数据，跳过
            t1_auc_volume_ratio = t1_auc_data["auc_volume_ratio_vs_5d_avg"].values[0]
            t1_order_book_ratio = t1_auc_data["order_book_volume_ratio"].values[0]
            
            # 3. T+1竞价二次筛选（仅保留早盘仍强势的个股）
            if t1_auc_volume_ratio < 1.1 or t1_order_book_ratio < 1.1:
                continue  # 竞价量比不足或盘口弱势，放弃买入
            
            # 4. 检查价格数据完整性（确保能完成交易）
            if t1_open is None or t2_open is None or t3_close is None:
                continue  # 关键价格缺失，跳过
            if t1_open <= 0 or t2_open <= 0 or t3_close <= 0:
                continue  # 无效价格，跳过
            
            # 5. 模拟交易（单只股票总投资10000元，分两次买入）
            total_invest = 10000  # 单只股票固定投资金额（可调整）
            # T+1开盘买入50%
            t1_invest = total_invest * 0.5
            t1_shares = t1_invest / t1_open  # 买入股数（不考虑手续费）
            # T+2开盘买入50%
            t2_invest = total_invest * 0.5
            t2_shares = t2_invest / t2_open
            # 总持股数
            total_shares = t1_shares + t2_shares
            
            # 6. 计算卖出收益（T+3收盘卖出）
            sell_amount = total_shares * t3_close  # 卖出总金额
            profit = sell_amount - total_invest  # 收益金额
            profit_ratio = (profit / total_invest) * 100  # 收益率（百分比）
            
            # 7. 记录交易详情
            trade_records.append({
                "选股日期": select_date,
                "股票代码": stock_code,
                "股票名称": stock_name,
                "所属行业": industry,
                "T+1开盘价": round(t1_open, 2),
                "T+2开盘价": round(t2_open, 2),
                "T+3收盘价": round(t3_close, 2),
                "T+1竞价量比": round(t1_auc_volume_ratio, 2),
                "T+1盘口量比": round(t1_order_book_ratio, 2),
                "总投入(元)": total_invest,
                "总卖出(元)": round(sell_amount, 2),
                "收益金额(元)": round(profit, 2),
                "收益率(%)": round(profit_ratio, 2)
            })
        
        # 检查是否有有效交易记录
        if not trade_records:
            print("无有效交易记录（可能所有个股均未通过T+1筛选或价格缺失）")
            return False
        
        # 8. 整理交易记录并保存
        self.trade_records = pd.DataFrame(trade_records)
        trade_save_path = os.path.join(self.performance_dir, "trade_records.csv")
        self.trade_records.to_csv(trade_save_path, index=False, encoding="utf-8-sig")
        
        # 9. 计算策略整体统计指标
        total_trades = len(self.trade_records)
        profitable_trades = (self.trade_records["收益率(%)"] > 0).sum()
        loss_trades = total_trades - profitable_trades
        zero_profit_trades = total_trades - profitable_trades - loss_trades
        
        # 核心统计指标
        self.overall_stats = {
            "回测周期": f"{self.trade_records['选股日期'].min().strftime('%Y-%m-%d')} 至 {self.trade_records['选股日期'].max().strftime('%Y-%m-%d')}",
            "总交易次数": total_trades,
            "盈利次数": profitable_trades,
            "亏损次数": loss_trades,
            "平本次数": zero_profit_trades,
            "胜率(%)": round((profitable_trades / total_trades) * 100, 2),
            "平均收益率(%)": round(self.trade_records["收益率(%)"].mean(), 2),
            "中位数收益率(%)": round(self.trade_records["收益率(%)"].median(), 2),
            "累计收益率(%)": round(((1 + self.trade_records["收益率(%)"]/100).prod() - 1) * 100, 2),
            "最大盈利(%)": round(self.trade_records["收益率(%)"].max(), 2),
            "最大亏损(%)": round(self.trade_records["收益率(%)"].min(), 2),
            "收益标准差(%)": round(self.trade_records["收益率(%)"].std(), 2),
            "单次最大收益金额(元)": round(self.trade_records["收益金额(元)"].max(), 2),
            "单次最大亏损金额(元)": round(self.trade_records["收益金额(元)"].min(), 2)
        }
        
        # 10. 计算行业表现统计
        industry_perf = self.trade_records.groupby("所属行业").agg({
            "收益率(%)": ["mean", "count", lambda x: (x > 0).mean() * 100],
            "收益金额(元)": "sum"
        }).reset_index()
        industry_perf.columns = ["所属行业", "平均收益率(%)", "交易次数", "胜率(%)", "总收益金额(元)"]
        industry_perf = industry_perf.sort_values("平均收益率(%)", ascending=False)
        
        # 11. 打印策略回测结果
        print(f"\n=== 策略回测结果汇总 ===")
        for key, value in self.overall_stats.items():
            print(f"{key}：{value}")
        
        print(f"\n=== 行业表现TOP3 ===")
        print(industry_perf.head(3).to_string(index=False, float_format=lambda x: f"{x:.2f}"))
        
        # 12. 保存完整统计结果（Excel格式，多sheet）
        with pd.ExcelWriter(os.path.join(self.performance_dir, "strategy_performance_stats.xlsx")) as writer:
            # 整体统计
            pd.DataFrame([self.overall_stats]).to_excel(writer, sheet_name="整体统计", index=False)
            # 行业表现
            industry_perf.to_excel(writer, sheet_name="行业表现", index=False)
            # 最佳/最差个股
            top10_stocks = self.trade_records.sort_values("收益率(%)", ascending=False).head(10)
            top10_stocks.to_excel(writer, sheet_name="最佳10只个股", index=False)
            bottom10_stocks = self.trade_records.sort_values("收益率(%)", ascending=True).head(10)
            bottom10_stocks.to_excel(writer, sheet_name="最差10只个股", index=False)
        
        print(f"\n=== 收益计算完成 ===")
        print(f"交易记录保存：{trade_save_path}")
        print(f"统计结果保存：{self.performance_dir}/strategy_performance_stats.xlsx")
        
        return True

    def plot_strategy_performance(self) -> None:
        """可视化策略表现（累计收益+每日收益+收益分布）"""
        if self.trade_records is None or self.overall_stats is None:
            print("无策略收益数据，无法生成可视化图表")
            return
        
        print(f"\n=== 生成策略表现可视化图表 ===")
        
        # 准备绘图数据
        # 1. 每日收益与累计收益数据
        daily_perf = self.trade_records.groupby("选股日期").agg({
            "收益率(%)": ["mean", "count"],
            "收益金额(元)": "sum"
        }).reset_index()
        daily_perf.columns = ["选股日期", "每日平均收益率(%)", "当日交易数", "当日总收益(元)"]
        # 计算累计收益率（复利）
        daily_perf = daily_perf.sort_values("选股日期")
        daily_perf["累计收益率(%)"] = ((1 + daily_perf["每日平均收益率(%)"]/100).cumprod() - 1) * 100
        
        # 2. 创建画布（3个子图：累计收益、每日收益、收益分布）
        fig = plt.figure(figsize=(16, 12))
        
        # 子图1：累计收益率曲线
        ax1 = plt.subplot(3, 1, 1)
        ax1.plot(daily_perf["选股日期"], daily_perf["累计收益率(%)"], 
                 color="#2E86AB", linewidth=2.5, label=f"累计收益率：{self.overall_stats['累计收益率(%)']}%")
        ax1.axhline(y=0, color="#A23B72", linestyle="--", alpha=0.8, linewidth=1.5)  # 零收益线
        ax1.fill_between(daily_perf["选股日期"], daily_perf["累计收益率(%)"], 0, 
                        where=(daily_perf["累计收益率(%)"] >= 0), color="#F18F01", alpha=0.3)
        ax1.fill_between(daily_perf["选股日期"], daily_perf["累计收益率(%)"], 0, 
                        where=(daily_perf["累计收益率(%)"] < 0), color="#C73E1D", alpha=0.3)
        ax1.set_title("策略累计收益率走势", fontsize=14, pad=20)
        ax1.set_ylabel("累计收益率(%)", fontsize=12)
        ax1.grid(True, alpha=0.3, linestyle="-", linewidth=0.5)
        ax1.legend(loc="upper left", fontsize=11)
        # 日期格式化
        ax1.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
        plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
        
        # 子图2：每日平均收益率与交易数
        ax2 = plt.subplot(3, 1, 2)
        # 双Y轴：左轴=每日平均收益率，右轴=当日交易数
        ax2_twin = ax2.twinx()
        
        # 每日平均收益率（柱状图）
        bars = ax2.bar(daily_perf["选股日期"], daily_perf["每日平均收益率(%)"], 
                       color="#6A994E", alpha=0.7, label="每日平均收益率(%)")
        ax2.axhline(y=0, color="#A23B72", linestyle="--", alpha=0.8, linewidth=1.5)
        ax2.set_ylabel("每日平均收益率(%)", fontsize=12, color="#6A994E")
        ax2.tick_params(axis="y", labelcolor="#6A994E")
        
        # 当日交易数（折线图）
        line = ax2_twin.plot(daily_perf["选股日期"], daily_perf["当日交易数"], 
                            color="#BC6C25", linewidth=2, marker="o", markersize=4, label="当日交易数")
        ax2_twin.set_ylabel("当日交易数", fontsize=12, color="#BC6C25")
        ax2_twin.tick_params(axis="y", labelcolor="#BC6C25")
        
        ax2.set_title("每日平均收益率与交易个股数", fontsize=14, pad=20)
        ax2.grid(True, alpha=0.3, linestyle="-", linewidth=0.5)
        # 日期格式化
        ax2.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
        plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)
        # 合并图例
        lines1, labels1 = ax2.get_legend_handles_labels()
        lines2, labels2 = ax2_twin.get_legend_handles_labels()
        ax2.legend(lines1 + lines2, labels1 + labels2, loc="upper right", fontsize=11)
        
        # 子图3：个股收益率分布直方图
        ax3 = plt.subplot(3, 1, 3)
        # 绘制直方图
        n, bins, patches = ax3.hist(self.trade_records["收益率(%)"], 
                                   bins=30, color="#264653", alpha=0.7, edgecolor="#2A9D8F", linewidth=0.5)
        # 标注平均收益率和零收益线
        ax3.axvline(x=self.trade_records["收益率(%)"].mean(), 
                    color="#E9C46A", linestyle="--", linewidth=2, label=f"平均收益率：{self.trade_records['收益率(%)'].mean():.2f}%")
        ax3.axvline(x=0, color="#E76F51", linestyle="-", linewidth=2, label="零收益线")
        ax3.set_title("个股收益率分布", fontsize=14, pad=20)
        ax3.set_xlabel("收益率(%)", fontsize=12)
        ax3.set_ylabel("个股数量", fontsize=12)
        ax3.grid(True, alpha=0.3, linestyle="-", linewidth=0.5)
        ax3.legend(loc="upper right", fontsize=11)
        
        # 调整子图间距
        plt.tight_layout(pad=3.0)
        
        # 保存图片
        plot_save_path = os.path.join(self.performance_dir, "strategy_performance_chart.png")
        plt.savefig(plot_save_path, dpi=300, bbox_inches="tight")
        plt.close(fig)
        
        print(f"可视化图表保存：{plot_save_path}")

# 主函数入口（直接运行）
if __name__ == "__main__":
    # ====================== 配置参数 ======================
    # 1. 数据路径（请改为你的实际数据路径，支持CSV或Parquet）
    DATA_PATH = r"D:\workspace\xiaoyao\data\factortable.parquet"  # 示例：r"D:\stock_data\factortable.parquet"
    # 2. 选股日期范围（请改为你的数据实际覆盖范围）
    START_DATE = "2025-01-01"
    END_DATE = "2025-09-30"
    # ======================================================
    
    # 创建全字段选股器实例
    stock_selector = FullFieldStockSelector(
        data_path=DATA_PATH,
        output_dir="selected_stocks_full_field",
        result_dir="selection_results_full",
        performance_dir="strategy_performance_full"
    )
    
    # 执行完整流程：数据加载→每日选股→收益计算→可视化
    print("=== 启动全字段选股与策略回测流程 ===")
    if stock_selector.load_and_preprocess_data():
        if stock_selector.run_daily_selection(start_date=START_DATE, end_date=END_DATE):
            if stock_selector.calculate_strategy_returns():
                stock_selector.plot_strategy_performance()
    
    print(f"\n=== 所有流程完成 ===")
    print(f"选股结果目录：{stock_selector.output_dir}")
    print(f"策略回测目录：{stock_selector.performance_dir}")

=== 启动全字段选股与策略回测流程 ===
加载全字段原始数据...
数据处理失败：[Errno 2] No such file or directory: 'sample.csv'

=== 所有流程完成 ===
选股结果目录：selected_stocks_full_field
策略回测目录：strategy_performance_full
